In [298]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [299]:
%run scripts/helper.py
%run scripts/models.py

In [300]:
crowd_train = load_file('./data/train.csv/train.csv', index_col='id')
crowd_test = load_file('./data/test.csv/test.csv', index_col='id')

In [273]:
# cross validation
train_index, test_index = ssSplit(crowd_train.median_relevance, train_size=8000, random_state=4)

In [274]:
# number of training examples
print len(train_index)

8000


In [275]:
# number of test examples
print len(test_index)

1016


In [276]:
# prepare training and validation set

# train = crowd_train.iloc[train_index]
# train_target = crowd_train.median_relevance.iloc[train_index].values
# validation = crowd_train.iloc[test_index]
# validation_target = crowd_train.median_relevance.iloc[test_index].values

In [301]:
## Prepare training on full training and test set

train = crowd_train
train_target = crowd_train.median_relevance.values
validation = crowd_test

In [302]:
train_text = tweak_text(train)
validation_text = tweak_text(validation)

### TFIDF on  full dataset rather than only on training set

In [294]:
# train_plus_test = []

# for i in range(len(train_text)):
#     train_plus_test.append(train_text[i])

# for i in range(len(validation_text)):
#     train_plus_test.append(validation_text[i])

In [295]:
# len(train_plus_test), ' ', len(train_text), ' ', len(validation_text)

### Non Linear Model using SVM

In [296]:
#(non_linear_model, tfv, svd, scl) = build_non_linear_model(train_text, train_target)

In [297]:
# print weighted kappa score on the training and test set


# training_preds_non = non_linear_model_predictions(non_linear_model, tfv, svd, scl, train_text)
# validation_preds_non = non_linear_model_predictions(non_linear_model, tfv, svd, scl, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_non, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_non, validation_target))

### Kneighbors model

In [283]:
(knn_model, tfv_knn, svd_knn) = build_knn_model(train_text, train_target)

In [284]:
# training_preds_knn = knn_model_predictions(knn_model, tfv_knn, svd_knn, train_text)
validation_preds_knn = knn_model_predictions(knn_model, tfv_knn, svd_knn, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_knn, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_knn, validation_target))

score on training set 0.6883 
#############################
score on test set 0.5294 


### Naive Bayes Model

In [285]:
# (bayes_model, tfv_bayes) = build_naive_bayes(train_text, train_target)

In [286]:
# training_preds_bayes = naive_bayes_predictions(bayes_model, tfv_bayes, train_text)
# validation_preds_bayes = naive_bayes_predictions(bayes_model, tfv_bayes, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_bayes, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_bayes, validation_target))

### Different model which tweaks stop words

In [287]:
# train_tweaked_text = tweak_text(train)
# validation_tweaked_text = tweak_text(validation)

In [288]:
# train_plus_test_tweak = []

# for i in range(len(train_tweaked_text)):
#     train_plus_test_tweak.append(train_tweaked_text[i])

# for i in range(len(validation_tweaked_text)):
#     train_plus_test_tweak.append(validation_tweaked_text[i])

In [289]:
(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked) = build_stopwords_tweak_model(train_text, train_target)

In [290]:
# training_preds_tweak = non_linear_model_predictions(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked, train_text)
validation_preds_tweak = non_linear_model_predictions(tweaked_model, tfv_tweaked, svd_tweaked, scl_tweaked, validation_text)

# print 'score on training set %0.4f ' %(quadratic_weighted_kappa(training_preds_tweak, train_target))
# print '#############################'
# print 'score on test set %0.4f ' %(quadratic_weighted_kappa(validation_preds_tweak, validation_target))

score on training set 0.8784 
#############################
score on test set 0.5333 


### Lets see how the ensemble of these different techniques work

In [291]:
pred_train = (training_preds_tweak + training_preds_knn ) / 2
pred_test = (validation_preds_tweak + validation_preds_knn ) / 2

print 'score on training set %0.4f ' %(quadratic_weighted_kappa(pred_train, train_target))
print '#############################'
print 'score on test set %0.4f ' %(quadratic_weighted_kappa(pred_test, validation_target))

score on training set 0.8294 
#############################
score on test set 0.5635 


In [105]:
make_submission(crowd_test.index.values.astype(int), pred_test, 'ensemble_knn.csv')